# Joint Analysis of HiPSCats

Here we will demonstrate the LSDB capabilities of analyzing multiple catalogs. LSDB comes equipped with its own nearest-neighbor crossmatching routine. 

In [1]:
from lsd2 import lsd2_client
client = lsd2_client()

2023-07-13 14:44:31,895	INFO worker.py:1636 -- Started a local Ray instance.
/astro/users/sdwyatt/anaconda3/envs/lsdb-test/lib/python3.10/site-packages/dask/config.py:694: UserWarning: Configuration key "shuffle" has been deprecated. Please use "dataframe.shuffle.algorithm" instead
  warnings.warn(


### First we start with opening two catalogs from a local registry

In [2]:
from lsd2 import registry as reg
HIPS_DIR ='/data3/epyc/projects3/ivoa_demo/'
lsdb_registry = reg.Registry(location=[HIPS_DIR])

Available HiPSCats:
   gaia
   des_y1a1
   ztf_dr14
   sdss


In [3]:
gaia = lsdb_registry.load_hipscat("gaia")
ztf = lsdb_registry.load_hipscat("ztf_dr14")

## Cross-matching

Let's cull our gaia catalog by performing a conesearch, then cross-match it with ztf, selecting a few columns

What our cross_match routine does is performs a nearest-neighbor (1) with a distance threshold of 0.1 degrees

In [4]:
gaia.cone_search(
    ra=30,
    dec=30,
    radius=5,
).cross_match(
    ztf,
    c1_cols=['ra', 'dec', 'pmra', 'pmdec'],
    c2_cols=['ps1_objid', 'nobs_g', 'nobs_r', 'nobs_i'],
    n_neighbors=1,
    dthresh=0.1, #degrees
).compute()

,gaia.ra,gaia.dec,gaia.pmra,gaia.pmdec,gaia.source_id,ztf_dr14.ps1_objid,ztf_dr14.nobs_g,ztf_dr14.nobs_r,ztf_dr14.nobs_i,ztf_dr14.ra,ztf_dr14.dec,hips_k,hips_pix,_DIST,_hipscat_index
75415,39.407850,29.919485,11.908070,-2.597590,128351249351236096,144010393792731429,0,8,5,39.379143,30.009038,4,59,0.092942,256702563125035008
75418,39.377382,29.912902,-2.439387,-15.872446,128351283710840576,144010393792731429,0,8,5,39.379143,30.009038,4,59,0.096148,256702610369675264
75419,39.400944,29.920950,8.546988,-11.095739,128351318070713216,144010393792731429,0,8,5,39.379143,30.009038,4,59,0.090090,256702653319348224
75420,39.424859,29.923626,20.928488,-18.433805,128351356725303040,144010393860043913,2,45,1,39.385962,30.011065,4,59,0.093707,256702722038824960
75421,39.426516,29.930165,NaN,NaN,128351386790190976,144010393860043913,2,45,1,39.385962,30.011065,4,59,0.088198,256702782168367104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587102,33.723268,19.538986,1.984760,-5.381827,99080428753094528,131350337551197365,150,263,0,33.755255,19.464005,3,8,0.080817,198160891865202688
587103,33.727912,19.549766,NaN,NaN,99080433048307840,131350337551197365,150,263,0,33.755255,19.464005,3,8,0.089550,198160921929973760
587104,33.730891,19.552507,0.752242,-0.520087,99080467408066560,131350337551197365,150,263,0,33.755255,19.464005,3,8,0.091433,198160969174614016
587105,33.715526,19.552119,NaN,NaN,99080501767805824,131350337551197365,150,263,0,33.755255,19.464005,3,8,0.095742,198161046484025344


### Performing a computation based off the xmatch result

Let's create a column based on the columns we selected from each catalog, and then reduce our result based on that column. Say, I want high-proper motion stars with `n_obs` greater than 50 in each ZTF photometric band. 
* I will assign a `pm` column 
* then query based off the `pm` column, and the `ztf_dr14.nobs_filter` columns
*   * need to use \` around columns with `column.name` notation

In [5]:
import numpy as np

gaia.cone_search(
    ra=30,
    dec=30,
    radius=5,
).cross_match(
    ztf,
    c1_cols=['ra', 'dec', 'pmra', 'pmdec'],
    c2_cols=['ps1_objid', 'nobs_g', 'nobs_r', 'nobs_i'],
    n_neighbors=1,
    dthresh=0.1, #degrees
).assign(
    pm=lambda x: np.sqrt(x['gaia.pmra']**2 + x['gaia.pmdec']**2),
).query(
    "pm > 100 and `ztf_dr14.nobs_g` > 50 and `ztf_dr14.nobs_r` > 50 and `ztf_dr14.nobs_i` > 50"
).compute()

,gaia.ra,gaia.dec,gaia.pmra,gaia.pmdec,gaia.source_id,ztf_dr14.ps1_objid,ztf_dr14.nobs_g,ztf_dr14.nobs_r,ztf_dr14.nobs_i,ztf_dr14.ra,ztf_dr14.dec,hips_k,hips_pix,_DIST,_hipscat_index,pm
166721,41.910463,32.412539,212.615433,-106.856004,130591573011485440,146960418600351329,480,690,52,41.860033,32.467302,4,59,0.069357,261183352181096448,237.956987
254702,39.989884,30.690740,102.874471,-72.991605,132902888188098432,144820399896889571,279,578,52,39.989994,30.690691,4,59,0.000106,265805574510215168,126.138539
255480,39.613570,30.814908,-487.539529,-386.861739,132922438878995968,144970396196792474,395,611,52,39.619670,30.809910,4,59,0.007240,265844727432085504,622.379946
257365,39.275207,30.931194,105.106276,-2.230016,132968270474211968,145110392750517992,428,691,52,39.275314,30.931210,4,59,0.000093,265936648322154496,105.129930
258622,40.059079,30.838724,75.514393,-73.233296,132998958016715136,145000400588967166,490,711,54,40.059148,30.838678,4,59,0.000075,265998126484029441,105.192867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166407,33.281305,17.561364,127.989898,-24.660190,80593137164436736,129070332810554199,66,715,57,33.281428,17.561370,3,8,0.000117,161186587860795392,130.343926
171589,33.562001,18.205236,297.171113,-100.413871,80832422677709568,129850335631094888,417,766,52,33.563137,18.211965,3,8,0.006815,161665240491098112,313.677567
173936,33.967456,18.562936,102.147559,38.594293,80939620765993088,130270339672616006,797,1243,55,33.967547,18.562992,3,8,0.000103,161879263006425088,109.195437
174044,33.684488,18.506677,106.563957,-51.158845,80944671647600768,130200336843098596,519,787,55,33.684589,18.506655,3,8,0.000099,161889686892052480,118.207886


In [6]:
client.shutdown()

/astro/users/sdwyatt/anaconda3/envs/lsdb-test/lib/python3.10/site-packages/dask/config.py:694: UserWarning: Configuration key "shuffle" has been deprecated. Please use "dataframe.shuffle.algorithm" instead
  warnings.warn(
